### Import Statements

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import re
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import datasets

### Spliting data into training and testing

In [28]:
def train_test_split(df, test_size=0.25, random_state=None):
    if random_state is not None:
        random.seed(random_state)
    
    if isinstance(test_size, float):
        test_size = int(test_size * df.shape[0])
        
    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)
    
    test_df = df.iloc[test_indices, :]
    train_df = df.drop(test_indices)
    
    return train_df, test_df

### Class node represents each node of the decision tree

In [29]:
class Node:
    def __init__(self, data, is_leaf=False):
        self.is_leaf = is_leaf                                       
        self.entropy = self.__calculate_entropy(data)                
        self.samples= np.unique(data[:, -1], return_counts=True)[1]  
        self.prediction = None                                       
        self.children = []
        self.question = None                                         
        self.gain_ratio = None                                       
        self.split_col = None
        self.split_col_name = None                                   
        
    def display(self):
        for i in range(len(self.samples)):
            if self.samples[i] > 0:
                print("Count of", i, "=", self.samples[i])

        print("Current Entropy is =", self.entropy)
        if self.is_leaf == False:
            print("Splitting on " + str(self.split_col_name), "with gain ratio", str(self.gain_ratio))
        else:
            print("Reached leaf node")
        print()
        
    def __calculate_entropy(self, data):
        counts = np.unique(data[:, -1], return_counts=True)[1]
        probabilities = counts / counts.sum()

        entropy = -sum(probabilities * np.log2(probabilities))
        if entropy == -0.0:
            entropy = 0.0
        return entropy

### Decision Tree class

In [39]:
class DecisionTree:
    def __init__(self, max_depth=None, categorical_threshold=0):
        self.root = None  
        self.max_depth = None
        self.categorical_threshold = 0
        
    # calculates entropy of a node
    def __calculate_entropy(self, data):
        counts = np.unique(data[:, -1], return_counts=True)[1]
        probabilities = counts / counts.sum()
        
        for probability in probabilities:
            if probability == 0:
                print("Probability 0 in tree calculate entropy")
                
        entropy = -sum(probabilities * np.log2(probabilities))
        if entropy == -0.0:
            entropy = 0.0
        return entropy
        
    # calculates overall entropy after split
    def __calculate_overall_entropy(self, split_data):
        overall_entropy = 0
        total = 0
        for data in split_data:
            total += len(data)
        
        for data in split_data:
            p_data = len(data) / total
            overall_entropy += p_data * self.__calculate_entropy(data)
            
        return overall_entropy
        
    # calculates gain_ratio of the split
    def __gain_ratio(self, node, split_data):
        info_gain = node.entropy - self.__calculate_overall_entropy(split_data)
        
        total = 0
        for data in split_data:
            total += len(data)
        
        split_number = 0
        
        for data in split_data:
            d_data = len(data) / total
            split_number += d_data * np.log2(d_data)

        gain_ratio = (-1)*(info_gain / split_number)
        return gain_ratio
        
    # returns the data after split
    def __split_data(self, data, split_column, split_value, feature_types):
        split_data = []
        if feature_types[split_column] == 'continuous':
            data_below = data[data[:, split_column] <= split_value]
            data_above = data[data[:, split_column] > split_value]
            split_data.append(data_below)
            split_data.append(data_above)
        else:
            for i in range(len(split_value)):
                new_data = data[data[:, split_column] == split_value[i]]
                split_data.append(new_data)
            
        return split_data
        
    # checks which split results in max decrease in entropy
    def __determine_best_split(self, data, potential_splits, feature_types):
        overall_entropy = 999

        for column_index in potential_splits:
            if feature_types[column_index] == 'continuous':
                for value in potential_splits[column_index]:
                    split_data = self.__split_data(data, column_index, value, feature_types)
                    curr_entropy = self.__calculate_overall_entropy(split_data)
                    if curr_entropy <= overall_entropy:
                        overall_entropy = curr_entropy
                        best_split_col = column_index
                        best_split_value = value
            else:
                split_data = self.__split_data(data, column_index, potential_splits[column_index], feature_types)
                curr_entropy = self.__calculate_overall_entropy(split_data)
                if curr_entropy <= overall_entropy:
                    overall_entropy = curr_entropy
                    best_split_col = column_index
                    best_split_value = potential_splits[column_index]

        return best_split_col, best_split_value
    
    # Potential splits is a dictionary {index of column : list of all possible split values for that column}
    def __get_potential_splits(self, data, feature_types, blacklist):
        potential_splits = {}

        for column_index in range(data.shape[1] - 1):
            if column_index in blacklist:
                continue
            if feature_types[column_index] == 'continuous':
                potential_splits[column_index] = []

                values = data[:, column_index]
                unique_values = np.unique(values)

                for index in range(1, (len(unique_values))):
                    current_value = unique_values[index]
                    previous_value = unique_values[index-1]

                    potential_split = (current_value + previous_value) / 2
                    potential_splits[column_index].append(potential_split)
            else:
                potential_splits[column_index] = np.unique(data[:, column_index]).tolist()
        return potential_splits
    
    # to check if there are any possible splits available
    def __is_splitting_possible(self, potential_splits):
        for k, v in potential_splits.items():
            if len(v) != 0:
                return True
        return False
    
    # Handling of categorical and continuous inputs
    def __determine_feature_types(self, df):
        feature_types = []
        n_unique_threshold = self.categorical_threshold
        
        for column in df.columns:
            unique_values = df[column].unique()
            value = unique_values[0]
            
            if isinstance(value, str) or len(unique_values) <= n_unique_threshold:
                feature_types.append('categorical')
            else:
                feature_types.append('continuous')
                
        return feature_types
    
    def __check_data_purity(self, data, potential_splits, level):
        if len(np.unique(data[:, -1])) == 1 or self.__is_splitting_possible(potential_splits) == False:
            return True
        if self.max_depth is not None and level > self.max_depth:
            return True
        return False
      
    # The public function to train the model
    def fit(self, df):
        feature_types = self.__determine_feature_types(df)
        self.root = self.__train_tree(df.values, df.columns, feature_types, blacklist=[])
        
    # Private function to train the model - required for recursion
    def __train_tree(self, data, columns=None, feature_types=None, blacklist=[], level=0):
        node = Node(data)                        
        
        potential_splits = self.__get_potential_splits(data, feature_types, blacklist)  
        
        # base case
        if self.__check_data_purity(data, potential_splits, level):
            node.is_leaf = True
            classes, counts = np.unique(data[:, -1], return_counts=True)
            node.prediction = classes[counts.argmax()]
            return node
        
        node.is_leaf = False
        split_col, split_val = self.__determine_best_split(data, potential_splits, feature_types) 
        split_data = self.__split_data(data, split_col, split_val, feature_types)
            
        # Updating parameters of node
        node.gain_ratio = self.__gain_ratio(node, split_data)
        node.split_col = split_col
        node.split_col_name = columns[split_col]
        
        if feature_types[split_col] == 'continuous':
            node.question = str(split_col) + " <= " + str(split_val)
        else:
            node.question = str(split_col) + " == " + str(split_val)
            blacklist.append(split_col)
         
        for split in split_data:
            node.children.append(self.__train_tree(split, columns, feature_types, blacklist, level+1))
            
        return node  
    
    def display(self, node=None, level=0):
        if node is None:
            node = self.root
            
        print("Level", level)
        node.display()
        
        for i in range(len(node.children)):
            self.display(node.children[i], level+1)
    
    
    def predict(self, df_test):
        feature_types = self.__determine_feature_types(df_test)
        data = df_test.values
        rv = []
        
        for arr in data:
            classification = self.__classify(arr, feature_types)
            rv.append(classification)
            
        return rv
        
    
    def __classify(self, arr, feature_types=None, node=None):
        if node is None:
            node = self.root
            
        if node.is_leaf == True:
            return node.prediction
        else:
            question = node.question
            col, _, vals = question.split(" ", maxsplit=2)
            
            if feature_types[int(col)] == 'continuous':
                if arr[int(col)] <= float(vals):   
                    return self.__classify(arr, feature_types, node.children[0])
                else:
                    return self.__classify(arr, feature_types, node.children[1])
            else:
                vals = vals.strip('][').split(', ')
                i = 0
                for val in vals:
                    val = val.strip("\'\''")
                    if arr[int(col)] == val:   
                        return self.__classify(arr, feature_types, node.children[i])
                    i += 1

    # accuracy = total correct perdictions / total predictions
    def accuracy(self, y_pred, y_test):        
        is_correct = (y_pred == y_test)
        return len(is_correct[is_correct == True]) / len(y_test)

### Load dataset

In [51]:
mnist = datasets.load_digits()
data = mnist.data
df = pd.DataFrame(data)
df['label'] = mnist.target
train_df, test_df = train_test_split(df, random_state=1)

In [52]:
clf = DecisionTree(categorical_threshold=0)
clf.fit(train_df)
clf.display()

Level 0
Count of 0 = 126
Count of 1 = 135
Count of 2 = 136
Count of 3 = 131
Count of 4 = 138
Count of 5 = 141
Count of 6 = 134
Count of 7 = 130
Count of 8 = 134
Count of 9 = 143
Current Entropy is = 3.3210000547771013
Splitting on 42 with gain ratio 0.4713259538414555

Level 1
Count of 0 = 3
Count of 1 = 87
Count of 2 = 91
Count of 3 = 123
Count of 4 = 34
Count of 5 = 134
Count of 6 = 1
Count of 7 = 86
Count of 8 = 28
Count of 9 = 141
Current Entropy is = 2.8797716957070936
Splitting on 26 with gain ratio 0.42465396379030956

Level 2
Count of 0 = 3
Count of 1 = 46
Count of 2 = 83
Count of 3 = 116
Count of 4 = 8
Count of 5 = 1
Count of 6 = 64
Count of 7 = 2
Count of 8 = 47
Current Entropy is = 2.4379801584994794
Splitting on 43 with gain ratio 0.593775692653168

Level 3
Count of 0 = 3
Count of 1 = 13
Count of 2 = 7
Count of 3 = 103
Count of 4 = 1
Count of 5 = 2
Count of 6 = 47
Current Entropy is = 1.639617308290017
Splitting on 30 with gain ratio 0.6315085520358464

Level 4
Count of 0 =

Count of 3 = 19
Count of 4 = 1
Count of 5 = 12
Current Entropy is = 2.0342159065712604
Splitting on 13 with gain ratio 0.6081269301973055

Level 5
Count of 0 = 1
Count of 1 = 15
Count of 2 = 1
Current Entropy is = 0.6402064333604701
Splitting on 50 with gain ratio 0.9999999999999998

Level 6
Count of 0 = 15
Current Entropy is = 0.0
Reached leaf node

Level 6
Count of 0 = 1
Count of 1 = 1
Current Entropy is = 1.0
Splitting on 60 with gain ratio 1.0

Level 7
Count of 0 = 1
Current Entropy is = 0.0
Reached leaf node

Level 7
Count of 0 = 1
Current Entropy is = 0.0
Reached leaf node

Level 5
Count of 0 = 5
Count of 1 = 4
Count of 2 = 19
Count of 3 = 1
Count of 4 = 12
Current Entropy is = 1.8614589988005519
Splitting on 12 with gain ratio 0.5799741597110105

Level 6
Count of 0 = 5
Count of 1 = 2
Count of 2 = 1
Count of 3 = 4
Current Entropy is = 1.7841591278514217
Splitting on 43 with gain ratio 0.9999999999999999

Level 7
Count of 0 = 2
Count of 1 = 1
Count of 2 = 4
Current Entropy is = 1.

Count of 0 = 1
Current Entropy is = 0.0
Reached leaf node

Level 5
Count of 0 = 3
Count of 1 = 92
Count of 2 = 1
Count of 3 = 1
Current Entropy is = 0.36359750755404885
Splitting on 43 with gain ratio 0.8854505974407747

Level 6
Count of 0 = 2
Count of 1 = 1
Count of 2 = 1
Current Entropy is = 1.5
Splitting on 60 with gain ratio 1.0

Level 7
Count of 0 = 1
Count of 1 = 1
Current Entropy is = 1.0
Splitting on 61 with gain ratio 1.0

Level 8
Count of 0 = 1
Current Entropy is = 0.0
Reached leaf node

Level 8
Count of 0 = 1
Current Entropy is = 0.0
Reached leaf node

Level 7
Count of 0 = 2
Current Entropy is = 0.0
Reached leaf node

Level 6
Count of 0 = 1
Count of 1 = 92
Current Entropy is = 0.0857426825355025
Splitting on 53 with gain ratio 0.42878793028240564

Level 7
Count of 0 = 91
Current Entropy is = 0.0
Reached leaf node

Level 7
Count of 0 = 1
Count of 1 = 1
Current Entropy is = 1.0
Splitting on 61 with gain ratio 1.0

Level 8
Count of 0 = 1
Current Entropy is = 0.0
Reached leaf no

In [53]:
y_pred = clf.predict(test_df.iloc[:, :-1])

In [54]:
clf.accuracy(y_pred=y_pred, y_test=test_df.values[:, -1])

0.8708240534521158

In [55]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(train_df.values[:, :-1], train_df.values[:, -1])
clf.score(test_df.values[:, :-1], test_df.values[:, -1])

0.8285077951002228